# Lesson: Towards Evaluating The Differential Privacy of a Function

Intuitively, we want to be able to query our database and evaluate whether or not the result of the query is leaking "private" information. As mentioned previously, this is about **evaluating whether the output of a query changes** when we **remove someone** from the database. Specifically, we want to evaluate the **maximum** amount the query **changes** when someone is removed (maximum over all possible people who could be removed). So, in order to evaluate how much privacy is leaked, we're going to iterate over each person in the database and measure the difference in the output of the query relative to when we query the entire database.

Just for the sake of argument, let's make our first "database query" a simple sum. Aka, we're going to count the number of 1s in the database.

In [1]:
import torch

In [2]:
def get_parallel_db(db, remove_idx):
    return torch.cat((db[:remove_idx], db[remove_idx+1:]))

In [3]:
def get_parallel_dbs(db):
    dbs = []
    for remove_idx in range(len(db)):
        dbs.append(get_parallel_db(db, remove_idx))
    return dbs

In [4]:
def create_db_and_parallels(num_entries):
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    return db, pdbs

In [5]:
db, pdbs = create_db_and_parallels(5000)

In [6]:
def query(db):
    return db.sum()

In [7]:
full_db_result = query(db)

In [12]:
full_db_result

tensor(2535)

In [14]:
query(pdbs[16])

tensor(2534)

**sensitivity**, or **L1 sensitivity**: The **Maximum** amount that the query changes when removing an individual from the database

In [9]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    db_distance = torch.abs(full_db_result - pdb_result).item()
    if db_distance > sensitivity:
        sensitivity = db_distance

In [10]:
sensitivity

1

The output of the **sum** is conditioned on **every** individual that is a **1** in the database

## Project - Evaluating the Privacy of a Function
In the last section, we measured the difference between each parallel db's query result and the query result for the entire database and then calculated the max value (which was 1). This value is called "sensitivity", and it **corresponds to the function** we chose for the query. Namely, the "sum" query will always have a sensitivity of exactly 1. However, we can also calculate sensitivity for other functions as well.

Let's try to calculate sensitivity for the "mean" function.

In [36]:
def get_sensitivity(query, n_entries=1000):
    db, pdbs = create_db_and_parallels(n_entries)
    full_db_result = query(db)
    sensitivity = 0
    for pdb in pdbs:
        pdb_result = query(pdb)
        db_distance = torch.abs(full_db_result - pdb_result).item()
        if db_distance > sensitivity:
            sensitivity = db_distance
    return sensitivity

In [37]:
def query(db):
    return db.float().mean()

In [38]:
get_sensitivity(query)

0.0005095005035400391

In [35]:
0.5 / 1000 

0.0005

Wow! That sensitivity is WAY lower. Note the intuition here. "Sensitivity" is measuring how sensitive the output of the query is to **a person being removed from the database**. For a simple sum, this is always 1, but for the **mean**, removing a person is going to change the result of the query by rougly $\frac{1}{|db|}$ (which is much smaller). Thus, "mean" is a VASTLY **less "sensitive"** function (query) than SUM.

# Project: Calculate L1 Sensitivity For Threshold

In this first project, I want you to calculate the sensitivty for the "threshold" function.

- First compute the **sum over the database** (i.e. sum(db)) and return whether that **sum is greater than a certain threshold**.
- Then, I want you to create databases of size 10 and threshold of 5 and calculate the sensitivity of the function.
- Finally, re-initialize the database 10 times and calculate the sensitivity each time.

In [52]:
def threshold_fn(db, threshold=5):
    """
    Return whether the sum of the db is greater than a certain threshold
    """
    return (db.sum() > threshold).float()

In [55]:
for i in range(10):
    print(get_sensitivity(threshold_fn, 10))

0
1.0
1.0
0
1.0
0
0
1.0
0
1.0


If **sum** of the database is **exactly 6**, the sensitivity will be __1__, otherwise it will be **0**

We can see that sensitivity:
- Sometimes it depends only on the function and potential range of the data
- Sometimes it depends on specific data, a.k.a **data conditioned sensitivity**, sensitivity of different databases has different values

## Lesson: A Basic Differencing Attack

Sadly none of the functions we've looked at so far are differentially private (despite them having varying levels of sensitivity). The most basic type of attack can be done as follows.

Let's say we wanted to figure out a specific person's value in the database. All we would have to do is query for the sum of the entire database and then the sum of the entire database without that person!

## Project: Perform a Differencing Attack on Row 10

In this project, I want you to **construct a database** and then demonstrate how you can **use two different sum queries** to **explose** the value of the person represented by row 10 in the database (note, you'll need to use a database with at least 10 rows)

In [66]:
db = torch.randn(100) > 0.5

In [67]:
db[10]

tensor(1, dtype=torch.uint8)

In [71]:
sum(db)

tensor(33, dtype=torch.uint8)

In [68]:
db_exclude_row_10 = get_parallel_db(db, 10)

In [69]:
# differencing attack using sum query
sum(db) - sum(db_exclude_row_10) # value of 10th row

tensor(1, dtype=torch.uint8)

In [70]:
# differencing attack using mean query
sum(db).float() / len(db) - sum(db_exclude_row_10).float() / len(db_exclude_row_10)

tensor(0.0068)

In [73]:
# differencing attack using threshold query
(sum(db).float() > 32) - (sum(db_exclude_row_10).float() > 33)

tensor(1, dtype=torch.uint8)

They're all performing a query with a value that's missing, and **all the results are not 0**, mean that **part of the information in row 10 are leaking**